### 阶段1：基于生成器的协程（Python 2.5+）

In [1]:
def simple_generator():
    yield 1
    x = yield 2  
    yield x + 3

gen = simple_generator()
print(next(gen))  # 1
print(gen.send(10))  # 2
print(gen.send(10))  # 13
print(next(gen))  # StopIteration

1
2
13


StopIteration: 

### 阶段2：@asyncio.coroutine装饰器（Python 3.4）

In [ ]:
import asyncio

# 协程雏形 (Python 3.4) ，在Python 3.12中，@asyncio.coroutine装饰器已被移除
@asyncio.coroutine
def old_style_coro():
    print("Start")
    yield from asyncio.sleep(1)  # 委托给另一个生成器
    print("End")

# 等价于：
def old_style_coro_expanded():
    print("Start")
    # yield from展开为显式循环
    sleep_gen = asyncio.sleep(1)
    try:
        while True:
            value = next(sleep_gen)
            try:
                yielded_value = yield value
            except Exception as e:
                sleep_gen.throw(e)
            else:
                sleep_gen.send(yielded_value)
    except StopIteration:
        pass
    print("End")

AttributeError: module 'asyncio' has no attribute 'coroutine'

### 阶段3：原生协程（Python 3.5+）

In [3]:
async def native_coro():
    print("Start")
    await asyncio.sleep(1)  # 语法糖替代 yield from
    print("End")

# 反编译查看字节码差异
import dis
dis.dis(native_coro)

  1           0 RETURN_GENERATOR
              2 POP_TOP
              4 RESUME                   0

  2           6 LOAD_GLOBAL              1 (NULL + print)
             16 LOAD_CONST               1 ('Start')
             18 CALL                     1
             26 POP_TOP

  3          28 LOAD_GLOBAL              2 (asyncio)
             38 LOAD_ATTR                5 (NULL|self + sleep)
             58 LOAD_CONST               2 (1)
             60 CALL                     1
             68 GET_AWAITABLE            0
             70 LOAD_CONST               0 (None)
        >>   72 SEND                     3 (to 82)
             76 YIELD_VALUE              2
             78 RESUME                   3
             80 JUMP_BACKWARD_NO_INTERRUPT     5 (to 72)
        >>   82 END_SEND
             84 POP_TOP

  4          86 LOAD_GLOBAL              1 (NULL + print)
             96 LOAD_CONST               3 ('End')
             98 CALL                     1
            106 POP_TOP
 